In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = "/content/drive/MyDrive/YandexCup/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Загружаем скачанный классификатор токсичности:

#Score

In [ ]:
!pip install -q transformers
!pip install -q natasha
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru
!pip install -q hnswlib
!pip install -q pymystem3
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-10-18 08:12:01--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.243, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-10-18 08:12:01--  http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)... 5.45.205.221, 2a02:6b8::3:221
Connecting to cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)|5.45.205.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.3’

mystem-3.0-linux3.1 100%[===================>]  15.70M 

In [ ]:
import gensim
from pymystem3 import Mystem

stemmer = Mystem()
lemmas = stemmer.lemmatize("Красивая мама красиво мыла раму")
print(lemmas)

['красивый', ' ', 'мама', ' ', 'красиво', ' ', 'мыть', ' ', 'рама', '\n']


In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install -q kenlm

  Using cached https://github.com/kpu/kenlm/archive/master.zip (540 kB)


In [ ]:
import numpy as np

import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import softmax, sigmoid
from tqdm.auto import tqdm
import argparse
from pymystem3 import Mystem
import sys
from collections import Counter
from functools import partial
import kenlm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta")

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta").cuda()

lm = kenlm.Model('/content/drive/MyDrive/YandexCup/lm.binary')



TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()


def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch


def predict_toxicity_1(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda'):
    results = []
    for batch in iterate_batches(sentences, batch_size):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
    
        logits = model.to(device)(**{key: val.to(device) for key, val in tokenized.items()}).logits
        preds = logits_to_toxic_probas(logits)
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results


def get_w2v_indicies_1(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append((w, embs_voc[w]))
        else:
            for lemma in stemmer.lemmatize(w):
                if lemma.isalpha():
                    res.append((w, embs_voc.get(lemma, None)))
    return res


def load_embeddings(path):
    embs_file = np.load(path, allow_pickle=True)
    embs_vectors = embs_file['vectors']
    embs_voc = embs_file['voc'].item()

    embs_voc_by_id = [None for i in range(len(embs_vectors))]
    for word, idx in embs_voc.items():
        if embs_voc_by_id[idx] is None:
            embs_voc_by_id[idx] = word
    return embs_vectors, embs_voc, embs_voc_by_id


def calc_embs_1(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies_1(words)
    return [(w, i if i is None else embs_vectors[i]) for w, i in inds]


def calc_single_embedding_dist(a, b):
    a_s, a_v = a  #слово, вектор
    b_s, b_v = b  #слово, вектор
    if a_s == b_s: #если слова равны
        return 0.0
    if a_v is None or b_v is None: #если одно из векторов нет - дать пизды 
        return 1.0
    a = a_v
    b = b_v
    # inexact match is punished by 0.1
    return 0.1 + 0.9 * (1 - a.dot(b) / np.linalg.norm(a) / np.linalg.norm(b)) / 2


def greedy_match_embs(a, b, max_dist=99999, cache=None, a_ind=0, b_ind=0):
    a_len = len(a) - a_ind #сколько слов мы еще не рассмотрели в а
    b_len = len(b) - b_ind #сколько слов мы еще не рассмотрели в b
    minlen = min(a_len, b_len) #минимальная длина строки
    maxlen = max(a_len, b_len) #минимальная длина строки
    if minlen == 0: 
        return np.minimum(maxlen, max_dist) 
    if maxlen - minlen >= max_dist: 
        return max_dist 
    
    if cache is None:
        cache = {}
    
    cache_key = (a_len, b_len)
    if cache_key in cache:
        return cache[cache_key]
        
    min_dist = max_dist
    
    first_dist = calc_single_embedding_dist(a[a_ind], b[b_ind])
    if max_dist >= first_dist:
        min_dist = np.minimum(min_dist, first_dist + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind + 1
        ))
    
    if first_dist > 0 and max_dist >= 1:
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind
        ))
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind, b_ind + 1
        ))
    
    cache[cache_key] = min_dist
    
    return min_dist



def calc_semantic_distance(a, b):
    a_embs = calc_embs_1(a)
    b_embs = calc_embs_1(b)
    
    clip_distance = 5  # this clips long computations
    return np.exp(-(greedy_match_embs(a_embs, b_embs, max_dist=clip_distance) / (0.6 * np.log(1 + len(a)))) ** 2)


def distance_score(original, fixed):
    original = original.split()
    fixed = fixed.split()
    
    return calc_semantic_distance(original, fixed)


def compute_lmdiff(original, fixed):
    original_lm_logproba = lm.score(original, bos=True, eos=True)
    fixed_lm_logproba = lm.score(fixed, bos=True, eos=True)
    
    probability_fraction = 10**((fixed_lm_logproba - original_lm_logproba) / 25)
    
    return np.clip(probability_fraction, 0.0, 1.0)


def compute_score(original_sentences, fixed_sentences, threshold=0.5, batch_size=5, show_res = False, return_result = False):
    fixed_toxicities = predict_toxicity_1(fixed_sentences, threshold=threshold, batch_size=batch_size, return_scores=True)
    scores = []
    lmdiffs = []
    emb_dists = []
    for original_sentence, fixed_sentence, fixed_toxicity in tqdm(zip(
        original_sentences, fixed_sentences, fixed_toxicities
    ), miniters=250, disable= True):
        original_sentence = normalize(original_sentence)
        fixed_sentence = normalize(fixed_sentence)
        
        distance = distance_score(original_sentence, fixed_sentence)
        lmdiff = compute_lmdiff(original_sentence, fixed_sentence)
        
        score = (1 - fixed_toxicity) * distance * lmdiff
        
        lmdiffs.append(lmdiff)
        emb_dists.append(distance)
        scores.append(score)
        
    if show_res:
        print('average toxicity:', np.mean(fixed_toxicities), file=sys.stderr)
        print('mean lmdiff:', np.mean(lmdiffs), file=sys.stderr)
        print('mean distance_score:', np.mean(emb_dists), file=sys.stderr)
        print('score', np.mean(scores) * 100, file=sys.stderr)
        
    if return_result:
        return {"scores" : scores, 
                "toxicities" : fixed_toxicities, 
                "lmdiffs" : lmdiffs,
                "distance_scores" : emb_dists}
    else:
        return scores

'''
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('original_texts', type=argparse.FileType('r'))
    parser.add_argument('fixed_texts', type=argparse.FileType('r'))
    parser.add_argument('--score', type=argparse.FileType('w'))
    parser.add_argument('--model', required=True, type=str)
    parser.add_argument('--embeddings', type=str, required=True)
    parser.add_argument('--lm', type=str, required=True)
    parser.add_argument('--device', type=str, choices=['cuda', 'cpu'], default='cpu')
    
    return parser.parse_args()
'''

def new_checker(original_texts, fixed_texts, device='cuda', show_res = False, return_result = False):
    original_texts = list(map(str.strip, original_texts))
    fixed_texts = list(map(str.strip, fixed_texts))
    
    assert len(original_texts) == len(fixed_texts)
    
    with torch.inference_mode(True):
        if return_result:
            return compute_score(original_texts, fixed_texts, show_res = show_res, return_result=return_result)
        return compute_score(original_texts, fixed_texts, show_res = show_res)


#Solution:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained(DRIVE_PATH + "trained_roberta")

model = AutoModelForSequenceClassification.from_pretrained(DRIVE_PATH + "trained_roberta").cuda()

TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


Ниже функции для применения классификатора

In [ ]:
from torch import softmax, sigmoid
import numpy as np


import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm.auto import tqdm
import argparse
from pymystem3 import Mystem
import sys
from collections import Counter
from functools import partial



ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()


def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

from tqdm.auto import tqdm
def predict_toxicity(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda', off_tqdm=False):
    results = []
    tqdm_fn = tqdm if verbose else lambda x, total: x
    for batch in tqdm_fn(iterate_batches(sentences, batch_size), total=np.ceil(len(sentences) / batch_size),  disable=off_tqdm):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
        
        model_input = {key: val.to(device) for key, val in tokenized.items()}
        
        logits = model.to(device)(**model_input).logits
        
        preds = logits_to_toxic_probas(logits)
        
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results


In [ ]:
texts = []
raw_texts = []
with open(DRIVE_PATH + 'public_testset.txt', 'rt') as f:
    for line in f:
        raw_texts.append(line) 
        texts.append(normalize(line))

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


Вычисляем токсичность отдельных слов

In [ ]:
line_toxicities = []
line_mean_toxicities = []

for line in tqdm(np.random.choice(raw_texts, 25)):
    pred = predict_toxicity([line], return_scores=True, off_tqdm=True)
    
    line_words = normalize(line).split()
    toxi = [predict_toxicity([word], return_scores=True, off_tqdm=True) for word in line_words]
    
    line_toxicities.append(pred[0])
    line_mean_toxicities.append(np.mean(toxi))



  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
print(f"На ({len(line_toxicities)} комментах):")
print(np.mean(line_toxicities), f"- средняя токсичность всего комментария")
print(np.mean(line_mean_toxicities), "- cредняя токсичность каждого слова в комментарии")

На (25 комментах):
0.7006662 - средняя токсичность всего комментария
0.202212 - cредняя токсичность каждого слова в комментарии


In [ ]:
line = raw_texts[100]
pred = predict_toxicity([line], return_scores=True, off_tqdm=True)

line_words = normalize(line).split()
toxi = [predict_toxicity([word], return_scores=True, off_tqdm=True) for word in line_words]

print(line)
print(pred)
print(line_words)
print(toxi)

Можно сказать проще - кукуха едет от старческого маразма.

[0.8303647]
['можно', 'сказать', 'проще', 'кукуха', 'едет', 'от', 'старческого', 'маразма']
[[0.07630243], [0.2619401], [0.04409236], [0.56395835], [0.15138271], [0.0645212], [0.2058756], [0.63614625]]


Ниже читаем эмбеддинги слов и описываем функции их обработки

In [ ]:
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru

from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
import pymorphy2


morph = pymorphy2.MorphAnalyzer(lang='ru')

In [ ]:
def save_comments(comments, name):
    with open(DRIVE_PATH + f'submits/{name}.txt', 'wt') as f:
        for text in comments:
            print(text, file=f)
        

In [ ]:
embs_file = np.load(DRIVE_PATH + 'embeddings_with_lemmas.npz', allow_pickle=True)
embs_vectors = embs_file['vectors']
embs_vectors_normed = embs_vectors / np.linalg.norm(embs_vectors, axis=1, keepdims=True)
embs_voc = embs_file['voc'].item()

embs_voc_by_id = [None for i in range(len(embs_vectors))]
for word, idx in embs_voc.items():
    if embs_voc_by_id[idx] is None:
        embs_voc_by_id[idx] = word

#Улучшаем словарь toxicity

In [ ]:
toxicity = dict()

In [ ]:
#словарь русских ругательств
def improve_toxicity(data):

    words = set()
    for text in data:
        words.update(text.split())
    words = sorted(words)

    with torch.inference_mode():
        word_toxicities = predict_toxicity(words, batch_size=100, return_scores=True)

    before = len(toxicity)
    for word, tox in zip(words, word_toxicities):
        toxicity[word] = tox
    print("New words :", len(toxicity) - before)
    print("Cur len :", len(toxicity))


In [ ]:
#1) Эмбеддинги
data = embs_voc.keys()
improve_toxicity(data)

  0%|          | 0/1627.0 [00:00<?, ?it/s]

New words : 162674
Cur len : 162674


In [ ]:
#2) Комментарии
data = texts
improve_toxicity(data)

  0%|          | 0/221.0 [00:00<?, ?it/s]

New words : 15015
Cur len : 177689


In [ ]:
#3) Словарь ругательств
data = []
with open(DRIVE_PATH + 'dicitionary/russkiy-mat.txt', 'rt') as f:
    for line in f:
        line = normalize(line)
        if line:
            data.append(line)

#improve_toxicity(data)

In [ ]:
#4) Комментарии из какого-то датасета
import pandas as pd
comments = pd.read_csv(DRIVE_PATH + "dicitionary/final.csv")

data = []
for line in tqdm(comments["text"].tolist()):
    line = normalize(line)
    if line:
        data.append(line)
        
#improve_toxicity(data)

  0%|          | 0/248287 [00:00<?, ?it/s]

In [ ]:
sorted_toxicity = {k: v for k, v in sorted(toxicity.items(), key=lambda item: item[1], reverse = True)}
for k, v in list(sorted_toxicity.items())[-100:]:
    print(k ,v)

cfd 0.009774271
http 0.009772634
print 0.009772043
8.cf 0.009771859
cf 0.009771859
account 0.009770866
sncf 0.0097690765
appstore 0.009768219
dcfta 0.009765147
title 0.009763957
call-center 0.009763764
correction 0.009762546
свеже 0.009760704
documents 0.009758518
sharepoint 0.009758454
listed 0.009756879
required 0.009756584
recommended 0.009754723
board 0.009749833
дизайновый 0.009749578
postscript 0.00974957
preview 0.009746988
pocketbook 0.009745045
wcf 0.009744373
openbox 0.009742492
закладка 0.009741862
types 0.009738403
winkhaus 0.0097380495
этoгo 0.009732593
yam 0.00973132
newsonline 0.009730524
onliner.by 0.009730259
mpls 0.009729734
allview 0.009727797
jcomments 0.009724541
schedule 0.009723984
callback 0.00972145
support 0.009715264
guestbook 0.009714621
аю 0.009714375
projector 0.009711683
cfg 0.009709422
order 0.009708761
speedo 0.009708757
н::спасибо 0.009708752
registration 0.009704544
provided 0.009703988
details 0.009703582
receive 0.009703357
purchase 0.009702743
proj

In [ ]:
def get_w2v_indicies(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append(embs_voc[w])
        else:
            
            lemma = stemmer.lemmatize(w)[0]#morph.parse(w)[0].normal_form
            res.append(embs_voc.get(lemma, None))
    return res

def calc_embs(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies(words)

    return [None if i is None else embs_vectors[i] for i in inds]

Сложим эмбеддинги нетоксичных слов в kd-дерево, чтобы можно было близко искать ближайших соседей

In [ ]:
normalized_embs_voc = {normalize(word) : ind for word, ind in tqdm(embs_voc.items()) if normalize(word)}
print(len(normalized_embs_voc))

  0%|          | 0/162690 [00:00<?, ?it/s]

148575


In [ ]:
MAX_TOXICITY  = 0.1

non_toxicity = {word : val for word, val in toxicity.items() if val <= MAX_TOXICITY}

nontoxic_vocab = {word : ind for word, ind in normalized_embs_voc.items() if toxicity.get(word, 1.0) <= MAX_TOXICITY}
nontoxic_emb_inds = {ind : word for word, ind in normalized_embs_voc.items() if toxicity.get(word, 1.0) <= MAX_TOXICITY}
embs_vectors_normed_nontoxic = [embs_vectors_normed[ind] for ind in nontoxic_emb_inds]

print(len(nontoxic_emb_inds))
print(len(embs_vectors_normed_nontoxic))

21111
21111


In [ ]:
#super non toxic
MAX_TOXICITY = 0.0097
super_non_toxic = {word for word in tqdm(normalized_embs_voc.keys()) if toxicity.get(word, 1.0) <= MAX_TOXICITY}
print(len(super_non_toxic))

  0%|          | 0/148575 [00:00<?, ?it/s]

44


#Теперь сокращаем словарь нетоксичных слов для брутфорса

In [ ]:
all_texts = ' '.join(texts)
all_texts = all_texts.split()
count = Counter(all_texts)
unique_words = set(all_texts)

Функция находит самое близкое нетоксичное слово по предпосчитанным эмбеддингам слов

In [ ]:
import hnswlib
import numpy as np
import pickle


num_elements, dim = embs_vectors_normed_nontoxic.shape

# Generating sample data
data = embs_vectors_normed_nontoxic
ids = np.arange(num_elements)

# Declaring index
p = hnswlib.Index(space = 'cosine', dim = dim) # possible options are l2, cosine or ip

# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 400, M = 16)

# Element insertion (can be called several times):
p.add_items(data, ids)

In [ ]:
#A better one

from functools import lru_cache

@lru_cache()
def find_closest_nontoxic(word, word_toxicity, allow_self=False):
    #if toxicity.get(word, 1.0) <= threshold:
    #    return word

    #word_lemmatized = morph.parse(word)[0].normal_form
    if word not in embs_voc:
        
            return word_toxicity, word
        #else:
           # word = word_lemmatized
    

    #threshold = min(toxicity.get(word, threshold), threshold)
    
    word_emb = calc_embs([word])
    if word_emb is None or word_emb[0] is None:
        return word_toxicity, word


    query = p.knn_query(word_emb, k = 200)[0][0]


    best_candidate = word
    best_score = word_toxicity

    

    for i in query:
        candidate = embs_voc_by_id[nontoxic_emb_inds[i]]
        similarity = distance_score(word, candidate)
        
        if (1 - toxicity.get(candidate)) + (similarity) > best_score:
            
            best_score = (1 - toxicity.get(candidate)) + (similarity)
            best_candidate = candidate 
            
    return toxicity.get(candidate), best_candidate


Заменяем токсичные слова на ближайшие по эмбеддингам не-токсичные

In [ ]:
def sort_by_toxicity(words):
    toxicities = [toxicity.get(w) if w in toxicity else predict_toxicity([w], return_scores = True, off_tqdm=True) for w in words]
  
    result = [[toxicities[i], i, words[i]] for i in range(len(words))]
    result.sort()
    return result

line = line = np.random.choice(raw_texts)
pred = predict_toxicity([line], return_scores=True, off_tqdm=True)

line_words = normalize(line).split()

print(sort_by_toxicity(line_words))

[[0.031997632, 3, 'мою'], [0.051976137, 7, 'тебе'], [0.05839065, 2, 'мы'], [0.09756993, 11, 'что'], [0.10556678, 8, 'сколько'], [0.116725855, 10, 'ты'], [0.11746098, 13, 'не'], [0.20037493, 15, 'белые'], [0.23048085, 12, 'краев'], [0.2553303, 14, 'видишь'], [0.26226667, 5, 'малость'], [0.2817571, 9, 'а'], [0.28510362, 1, 'щас'], [0.3707882, 16, 'росы'], [0.42197517, 4, 'тещу'], [0.60239124, 6, 'раскулачим'], [0.78310204, 0, 'федос']]


In [ ]:
def sort_by_distance(words, word):
    result = [[distance_score(word, words[i]), i, words[i]] for i in range(len(words))]
    result.sort()
    return result

print(texts[0])
sort_by_distance(texts[0].split(), "смерть")

он скоро сдохнет и все вернется


[[0.3301158710261462, 4, 'все'],
 [0.3321799419260301, 1, 'скоро'],
 [0.35243420745805787, 3, 'и'],
 [0.37890680578650543, 5, 'вернется'],
 [0.43616345093494785, 2, 'сдохнет'],
 [0.47781617663736303, 0, 'он']]

In [ ]:
def detox(line):
    
    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words) #[токсичность, индекс, слово]
    
    
    #cur_toxicity = predict_toxicity([' '.join(words)], return_scores=True, off_tqdm=True)[0]
  
    while sorted_words:
        toxic_toxicity, toxic_idx, toxic_word = sorted_words.pop()
        if toxic_toxicity > 0.77:
            fixed_word = "Спасибо"
        else:
            fixed_toxicity, fixed_word = find_closest_nontoxic(toxic_word, toxic_toxicity, allow_self = True)
        words[toxic_idx] = fixed_word
        #if fixed_word != toxic_word:
        #    sorted_words.append([fixed_toxicity, toxic_idx, fixed_word])
        #    sorted_words.sort()
        
        #cur_toxicity = predict_toxicity([' '.join(words)], return_scores=True, off_tqdm=True)[0]
        

    
    return ' '.join(words)

In [ ]:
limit = 0.3

candidates = set()
random_candidates = list(nontoxic_vocab.keys())
np.random.shuffle(random_candidates)

for candidate in tqdm(random_candidates):
    for word in candidates:
        similarity = distance_score(candidate, word)
        
        if similarity > limit:
            break
    else:
        candidates.add(candidate)

print(len(candidates))
candidates.update(super_non_toxic)
print(len(candidates))

  0%|          | 0/21266 [00:00<?, ?it/s]

65
109


In [ ]:
candidates = set()

In [ ]:
candidates.update(super_non_toxic)

In [ ]:

candidates.add("они")
candidates.add("любовь")
candidates.add("он")
candidates.add("textbook")
candidates.add("")
print(len(candidates))

49


In [ ]:
#Super non toxic
def find_most_similar(word):
    emb_word = calc_embs([word])
    max_similarity = 0
    result = "спасибо"
    for i in p.knn_query(emb_word, k = 5)[0][0]:
        
        other_word = embs_voc_by_id[super_nontoxic_emb_inds[i]]
        
        similarity = distance_score(word, other_word)
        if similarity > max_similarity:
            max_similarity = similarity
            result = other_word
        #print(other_word, similarity, toxicity.get(other_word))
    return result



def detox_brute(line):
    

    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words)
    
    while sorted_words:
        toxic_toxicity, toxic_idx, toxic_word = sorted_words.pop()
        

        original = ' '.join(words)
        candidate_sentences = [original]
        #### BRUTE FORCE
        for candidate in candidates:
            words[toxic_idx] = candidate
            candidate_sentences.append(' '.join(words))
        num_candidates = len(candidate_sentences)
        
        scores = np.array(new_checker([original] * num_candidates, candidate_sentences))
        words = candidate_sentences[np.argmax(scores)].split()
        

    
    return ' '.join(words)



In [ ]:
def random_letter(word):
    result_word = list(word)
    rand_letter = np.random.choice(list('абвгдежзийклмнопрстуфхцчшщъыьэюя'))
    rand_idx = np.random.randint(0, len(result_word))
    result_word[rand_idx] = rand_letter
    result_word = ''.join(result_word)
    return result_word

In [ ]:
def detox_dumb(i, line):
    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words)#sort_by_distance(words, "спасибо")
    
    score = new_checker([texts[i]], [' '.join(words)])
    for toxic_toxicity, toxic_idx, toxic_word in tqdm(sorted_words):
        #toxic_toxicity, toxic_idx, toxic_word = sorted_words#.pop()

        cur_word = toxic_word

        #RANDOM
        fixed_word = list(toxic_word)
        rand_letter = np.random.choice(list('абвгдежзийклмнопрстуфхцчшщъыьэюя'))
        rand_idx = np.random.randint(0, len(fixed_word))
        fixed_word[rand_idx] = rand_letter
        fixed_word = ''.join(fixed_word)
        candidates.add(fixed_word)
        


        for candidate in candidates:
            words[toxic_idx] = candidate
            new_score = new_checker([texts[i]], [' '.join(words)])

            if new_score >= score:
                print(score, "->", new_score, candidate)
                score = new_score
                cur_word = candidate
                
            else:
                words[toxic_idx] = cur_word
                #break
        candidates.remove(fixed_word)

    return ' '.join(words)

In [ ]:
cur_scores = []
def detox_letter(i, line):
    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words)#sort_by_distance(words, "спасибо")
    
    score = new_checker([texts[i]], [' '.join(words)])
    for toxic_toxicity, toxic_idx, toxic_word in sorted_words[::-1]:


        cur_word = toxic_word
        cands = ["спасибо", "textbook", "любовь", "productcode", "замечательный"]



        cands.append(random_letter(toxic_word))
        cands.append(random_letter(toxic_word))


        
        for candidate in cands:
            words[toxic_idx] = candidate
            new_score = new_checker([texts[i]], [' '.join(words)])

            if new_score > score:
                #print(score, "->", new_score, candidate)
                score = new_score
                cur_word = candidate
                
            else:
                words[toxic_idx] = cur_word
                #break

    cur_scores.append(score[0])
    return ' '.join(words)
    


In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-10-17 16:13:22--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.245, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-10-17 16:13:23--  http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)... 5.45.205.221, 2a02:6b8::3:221
Connecting to cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)|5.45.205.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.17’

mystem-3.0-linux3.1 100%[===================>]  15.70M

In [ ]:
Original = []
Detoxed = []
for i in tqdm(range(0, len(texts), 50)):
    line = texts[i]
    detoxed = detox_letter(i, line)

    Original.append(line)
    Detoxed.append(detoxed)


new_checker(Original, Detoxed, show_res = True)

In [ ]:
cur_scores = []
fixed_texts = []
for i in tqdm(range(len(texts))):
    fixed_texts.append(detox_letter(i, texts[i]))
    print(sum(cur_scores)/len(cur_scores), f"{i+1}/2500")

    save_comments(fixed_texts, "random_letter_fresh")

new_checker(texts, fixed_texts, show_res = True)

  0%|          | 0/2500 [00:00<?, ?it/s]

0.65157135468872 1/2500
0.5133867641885108 2/2500
0.5481759679670635 3/2500
0.5535967414406169 4/2500
0.5324004326822197 5/2500
0.49908806433776953 6/2500
0.49386850450281194 7/2500
0.506104684531847 8/2500
0.5115659221803718 9/2500
0.5163286124664827 10/2500
0.5408389111022713 11/2500
0.5601602098160946 12/2500
0.5931867194893464 13/2500
0.5903608557612079 14/2500
0.5974934858523535 15/2500
0.5945065183441136 16/2500
0.5849614616866772 17/2500
0.5677139900515447 18/2500
0.5803380316662117 19/2500
0.5787252404424683 20/2500
0.565813134155728 21/2500
0.5626180863606397 22/2500
0.5408574626393525 23/2500
0.5559634100103764 24/2500
0.5695066833954345 25/2500
0.5770733997266553 26/2500
0.57611204442378 27/2500
0.5715597824185638 28/2500
0.5649882790907843 29/2500
0.5729403713891046 30/2500
0.5754209049013412 31/2500
0.5765902188934621 32/2500
0.5709932372159918 33/2500
0.5622700281090374 34/2500
0.5635829022335483 35/2500
0.5695631244652697 36/2500
0.5639179341856515 37/2500
0.570695621827

#Сравнение решений:

In [ ]:
valya = []
kirill = []
count_valya = 0
count_kirill = 0 

with open(DRIVE_PATH + 'submits/bruteforce_hard_2_part.txt', 'rt') as f:
    for line in f:
        valya.append(line) 

with open(DRIVE_PATH + 'submits/full_thanks.txt', 'rt') as f:
    for line in f:
        kirill.append(line) 

super_result = []
for i in tqdm(range(2500)):
    kirill_score =  new_checker([texts[i]], [kirill[i]])
    valya_score = new_checker([texts[i]], [valya[i]])
    if valya_score > kirill_score:
        count_valya += 1
        super_result.append(valya[i])
        #print(valya_score,kirill_score, "Валя")
    else:
        count_kirill += 1
        super_result.append(kirill[i])
        #print(valya_score, kirill_score, "Кирилл")

    

print(count_valya, count_kirill)
new_checker(texts, super_result, show_res = True)

  0%|          | 0/2500 [00:00<?, ?it/s]

686 1814


average toxicity: 0.31801412
mean lmdiff: 0.9771276972705748
mean distance_score: 0.755816421040236


51.008772758835654

#Merges solutions into ultimate one

In [ ]:
ultimate_solution = []

names = os.listdir(path="/content/drive/MyDrive/YandexCup/submits_pro")
if '.ipynb_checkpoints' in names:
    names.remove('.ipynb_checkpoints')


solutions = {name : [] for name in names}
count = {name : 0 for name in names}
#reading submits
for name in names:
    sol = []
    with open(DRIVE_PATH + f'submits_pro/{name}', 'rt') as f:
        for line in f:
            sol.append(normalize(line))

    solutions[name] = sol[:]
    print(name, len(sol))

for i in tqdm(range(len(texts))):
    scores = {name : 0 for name in names}
    for name in names:
        scores[name] = new_checker([texts[i]], [solutions[name][i]])

    best_sol_name = max(scores, key=scores.get)
    count[best_sol_name] += 1
    ultimate_solution.append(solutions[best_sol_name][i])


print(count)
print(new_checker(texts, ultimate_solution, show_res = True))

SIXTY.txt 2500
brute_sixty.txt 2500


  0%|          | 0/2500 [00:00<?, ?it/s]

{'SIXTY.txt': 2205, 'brute_sixty.txt': 295}
[0.6515712991857765, 0.41035377513836024, 0.5839641164749069, 0.569859061861277, 0.4478327657071666, 0.3593172082155923, 0.4760463084606085, 0.622257708358335, 0.4784801003785525, 0.5451357906152633, 0.7859419161514587, 0.7142086275982319, 0.989504829980433, 0.5591575301445815, 0.7427940182619636, 0.5395135841503944, 0.37529397066803616, 0.3849129748510807, 0.7929211626328508, 0.5463535931512862, 0.32036160997659635, 0.6214584076353319, 0.07375318402061776, 0.9091440229201888, 0.8762345183643563, 0.7695760872195058, 0.6518566290418615, 0.5797071649522693, 0.3760679064670965, 0.8035510629415512, 0.6498369489217634, 0.6128389694401079, 0.3918898204006275, 0.31628119006580485, 0.6012832563992861, 0.7788709301022811, 0.37344814264505655, 0.8214700645780876, 0.45366952652729775, 0.7320285368961712, 0.8742081670674039, 0.6443624504316086, 0.666469911238444, 0.4069534569879465, 0.4753145401945713, 0.8826049270269579, 0.7788894034666167, 0.8401973109

average toxicity: 0.23484898
mean lmdiff: 0.9796217371116073
mean distance_score: 0.8013153201004338
score 60.57586762467172


In [ ]:
save_comments(ultimate_solution, "SIXTY_HALF")

# Сортируем по скору. Анализируем

In [ ]:
solution = []

with open(DRIVE_PATH + 'submits_pro/SIXTY_HALF.txt', 'rt') as f:
    for line in f:
        solution.append(normalize(line))


In [ ]:
result = new_checker(texts, solution, return_result = True)
print(result)
print(100*np.mean(result["scores"]))

{'scores': [0.6515712991857765, 0.41035377513836024, 0.5839641164749069, 0.569859061861277, 0.4478327657071666, 0.3593172082155923, 0.4760463084606085, 0.622257708358335, 0.4784801003785525, 0.5451357906152633, 0.7859419161514587, 0.7142086275982319, 0.989504829980433, 0.5591575301445815, 0.7427940182619636, 0.5395135841503944, 0.37529397066803616, 0.3849129748510807, 0.7929211626328508, 0.5463535931512862, 0.32036160997659635, 0.6214584076353319, 0.07375318402061776, 0.9091440229201888, 0.8762345183643563, 0.7695760872195058, 0.6518566290418615, 0.5797071649522693, 0.3760679064670965, 0.8035510629415512, 0.6498369489217634, 0.6128389694401079, 0.3918898204006275, 0.31628119006580485, 0.6012832563992861, 0.7788709301022811, 0.37344814264505655, 0.8214700645780876, 0.45366952652729775, 0.7320285368961712, 0.8742081670674039, 0.6443624504316086, 0.666469911238444, 0.4069534569879465, 0.4753145401945713, 0.8826049270269579, 0.7788894034666167, 0.8401973109355396, 0.5516834055114586, 0.070

In [ ]:
#sorted_by_score = [[score, i, solution[i]] for i, score in enumerate(result["scores"])]
sorted_by_score = [[toxicity, i, solution[i]] for i, toxicity in enumerate(result["toxicities"])]
sorted_by_score.sort(reverse=True)

skipped = 0
for score, i, comment in tqdm(sorted_by_score[60:100]):
    print(i, score, comment)
    if len(comment.split()) <= 80:
        solution[i] = detox_letter(i, comment)
    else:
        skipped += 1

  0%|          | 0/40 [00:00<?, ?it/s]

964 0.62829983 сталин всегда оставался сталиным он не для армян старался а для ссср любовь турции надо было преподать урок и то что наш нахиджеван и карабах оказался в составе любовь это тоже сталин у вас как не было так ее и нету своей древней истории и нечего воровать историю других народов то вы иранскую историю приспосабливаете под свою то турецкую ни у кого из исторических философоф ни в одной рукописях и манускриптах нет и слова азербаджан


BrokenPipeError: ignored

In [ ]:
save_comments(solution, "brute_sixty")

In [ ]:
new_checker(["чудило"], ["чудило"], show_res = True)

чудило


чудило




average toxicity: 0.9532205
mean lmdiff: 1.0
mean distance_score: 1.0
score 4.677951335906982


[0.046779513359069824]

##Для Online-решения

In [ ]:
print(len(nontoxic_emb_inds))
print(len(toxicity))

162690
162690


In [ ]:
import pickle as pkl

with open(DRIVE_PATH + 'data.pkl', 'wb') as f:
    pkl.dump(toxicity, f)
    

In [ ]:
len(toxicity)

368527

Сохраним данные для бейзлайна online-задачи